In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:00<00:00, 549MB/s]


In [2]:
image_source = '/home/temesgen_gebreabzgi/semantic_image_and_text_alignment/data/Challenge_Data/Assets/f16751048b64354c30b034c44981225f/endframe_2.png'

results = model.predict(image_source)


image 1/1 /home/temesgen_gebreabzgi/semantic_image_and_text_alignment/data/Challenge_Data/Assets/f16751048b64354c30b034c44981225f/endframe_2.png: 640x448 1 car, 1 stop sign, 1164.7ms
Speed: 4.2ms preprocess, 1164.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 448)


In [3]:
result = results[0]

In [4]:
len(result.boxes)

2

In [8]:
box = result.boxes[0]

In [9]:
print("Object type:", box.cls)
print("Coordinates:", box.xyxy)
print("Probability:", box.conf)

Object type: tensor([2.])
Coordinates: tensor([[417.0818, 609.0717, 598.7896, 745.2869]])
Probability: tensor([0.9477])


In [ ]:
print("Object type:",box.cls[0])
print("Coordinates:",box.xyxy[0])
print("Probability:",box.conf[0])

In [ ]:
cords = box.xyxy[0].tolist()
class_id = box.cls[0].item()
conf = box.conf[0].item()
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

YOLOv8 pretrained model, there are 80 object types with IDs from 0 to 79.

In [ ]:
print(result.names)

Let's modify the output to show results in a more representative way:

In [ ]:
cords = box.xyxy[0].tolist()
cords = [round(x) for x in cords]
class_id = result.names[box.cls[0].item()]
conf = round(box.conf[0].item(), 2)
print("Object type:", class_id)
print("Coordinates:", cords)
print("Probability:", conf)

In [ ]:
for box in result.boxes:
  class_id = result.names[box.cls[0].item()]
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2)
  print("Object type:", class_id)
  print("Coordinates:", cords)
  print("Probability:", conf)
  print("---")